In [1077]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import time
from time import sleep

import datetime

import matplotlib.pyplot as plt
from seaborn import set_style
set_style("whitegrid")

In [1078]:
soup = BeautifulSoup(requests.get('https://www.formula1.com/en/results.html').text, 'html.parser')
possible_years = ['https://www.formula1.com' + a['href'] 
                  for a in soup.find_all('a', attrs = {'data-name':'year'})]

year = [] 
race = []
driver = [] 

FP1_time = []
FP1_gap = []
FP1_lap = []

qualifying_time = []
grid_position = []

F1 = pd.DataFrame({'year':year, 'race':race, 
                   'driver':driver, 
                   'FP1 time':FP1_time,
                   'FP1 gap':FP1_gap,
                   'FP1 lap':FP1_lap,
                   'qualifying time':qualifying_time,
                   'grid position':grid_position})

## 1996 - 2024 29
for url in possible_years[:2]:
    html = requests.get(url)
    soup = BeautifulSoup(html.text,'html.parser')
    possible_races_url = ['https://www.formula1.com' + a['href'] 
                          for a in soup.find_all('a', attrs = {'data-name':'meetingKey'})][1:]
    possible_races = [x.split('/race-result.html')[0][57:] for x in possible_races_url]   
    for url in possible_races_url[:17]:
            FP1_url = url.replace('race-result','practice-1')
            FP1_html = requests.get(FP1_url)
            FP1_soup = BeautifulSoup(FP1_html.text,'html.parser')
            FP1_drivers = [x.text for x in FP1_soup.find_all('span', {'class':'hide-for-mobile'})]
            
            starting_grid_url = url.replace('race-result','starting-grid')
            starting_grid_html = requests.get(starting_grid_url)
            starting_grid_soup = BeautifulSoup(starting_grid_html.text,'html.parser')
            starting_grid = [x.text for x in starting_grid_soup.find_all('span', {'class':'hide-for-mobile'})]
            starting_grid_times = [starting_grid_soup.find_all('td', {'class':'dark bold'})[1 + 2 * (i - 1)].text 
                                   for i in range(1, len(starting_grid) + 1)]

            ## Find the drivers in FP1 with no grid data.
            FP1_cut_list = []
            FP1_cut_list_indices = []
            FP1_drivers_noGrid = [x for x in FP1_drivers if x in starting_grid]
            FP1_cut_list.extend(list(set(FP1_drivers) - set(FP1_drivers_noGrid)))
            for i in FP1_cut_list:
                FP1_cut_list_indices.extend([FP1_drivers.index(i)])

            ## Find the drivers on the grid with no FP1 data.
            grid_cut_list = []
            grid_cut_list_indices = []
            starting_grid_noFP1 = [x for x in starting_grid if x in FP1_drivers_noGrid]
            grid_cut_list.extend(list(set(starting_grid) - set(starting_grid_noFP1)))
            for i in grid_cut_list:
                grid_cut_list_indices.extend([starting_grid.index(i)])

            year.extend([url[41:45]] * len(FP1_drivers_noGrid))
            driver.extend(FP1_drivers_noGrid)
            race.extend([url.split('/race-result.html')[0][57:]] * len(FP1_drivers_noGrid))

            ## time
            FP1_time.extend([FP1_soup.find_all('td', {'class':'dark bold'})[1 + 3 * (i - 1)].text 
                             for i in range(1, len(FP1_drivers) + 1)])

            ## gap
            FP1_gap.extend([FP1_soup.find_all('td', {'class':'dark bold'})[1 + 3 * (i - 1) + 1].text[1:] 
                            for i in range(1, len(FP1_drivers) + 1)])
            FP1_gap[0] = 0

            ## lap
            FP1_lap.extend([FP1_soup.find_all('td', {'class':'bold'})[3 + 4 * (i - 1)].text 
                            for i in range(1, len(FP1_drivers) + 1)])

            ## Remove drivers from FP1.
            for i in FP1_cut_list_indices:
                del FP1_time[i:i + 1]
                del FP1_gap[i:i + 1]
                del FP1_lap[i:i + 1]

            ## Remove drivers from grid.
            for i in grid_cut_list_indices:
                del starting_grid_times[i:i + 1]

            ## Reorder the starting grid and qualifying times data to match the data frame.
            starting_grid_noFP1_copy = starting_grid_noFP1.copy() 
            starting_grid_times_copy = starting_grid_times.copy() 
            for i in range(0, len(FP1_drivers_noGrid)):
                index_position = starting_grid_noFP1_copy.index(FP1_drivers_noGrid[i])
                starting_grid_noFP1[i] = index_position + 1
                starting_grid_times[i] = starting_grid_times_copy[index_position]
            grid_position.extend(starting_grid_noFP1)
            qualifying_time.extend(starting_grid_times)

## Populate the data frame.
F1['year'] = year
F1['race'] = race
F1['driver'] = driver
F1['FP1 time'] = FP1_time
F1['FP1 gap'] = FP1_gap
F1['FP1 lap'] = FP1_lap
F1['qualifying time'] = qualifying_time
F1['grid position'] = grid_position

In [1079]:
## Replace missing data with NaN, and then use .dropna() to delete those rows.
F1 = F1.replace('',np.nan).dropna();
F1['FP1 gap'].replace('s','')
## Convert times to seconds
def convert(t):
    if ':' in t:
        sec_from_min = float(t.split(':')[0]) * 60
        sec = float(t.split(':')[1])
        return sec_from_min + sec
    else:
        return float(t)
F1['FP1 time'] = F1['FP1 time'].apply(convert)
F1['qualifying time'] = F1['qualifying time'].apply(convert)
F1['FP1 gap'] = [str(x).replace('s','') for x in F1['FP1 gap']]
F1['FP1 gap'] = F1['FP1 gap'].apply(convert)



In [1080]:
F1

,year,race,driver,FP1 time,FP1 gap,FP1 lap,qualifying time,grid position
0,2024,bahrain,Ricciardo,92.869,0.000,23,90.278,14
1,2024,bahrain,Norris,92.901,0.032,25,89.614,7
2,2024,bahrain,Piastri,93.183,0.314,24,89.683,8
3,2024,bahrain,Tsunoda,93.193,0.324,20,90.129,11
4,2024,bahrain,Alonso,93.251,0.382,25,89.542,6
...,...,...,...,...,...,...,...,...
440,2023,japan,Ocon,93.516,1.869,22,90.586,14
441,2023,japan,Hamilton,93.699,2.052,22,89.908,7
442,2023,japan,Bottas,93.860,2.213,24,91.049,16
443,2023,japan,Magnussen,93.975,2.328,23,90.665,15
